# Discounted Cash Flow model

## Part I: Generating the Data
The main idea is, we want to predict how much “free cash” a company will generate next year, the year after that, and so on. And because money today is worth more than money tomorrow (that’s why interest rates exist), we *discount* the money in future years before adding the results together.

You can read more on DCF models [here](https://www.investopedia.com/terms/d/dcf.asp) and [here](https://corporatefinanceinstitute.com/resources/knowledge/valuation/dcf-formula-guide/) and [here](https://sba.thehartford.com/finance/cash-flow/discounted-cash-flow/), or type the term into Google and find a million resources (I do recommend!)

For this lesson we’ll create a DCF model using financial data for Apple, maker of iPhones and iPads, and currently the largest company in the stock market (by market cap). To follow along manually, go to [stockrow](https://stockrow.com/) , type “AAPL” into the search bar, and navigate to the “Financials” section — specifically the “trailing” section which is short for Trailing 12 Months.

Download (or “Export”) the .xlsx files for the Income Statement (TTM), Cash Flow statement (TTM), and Balance Sheet (Quarterly)

I had previously downloaded these Excel workbooks to my local machine. The following python code will quickly format the data so that it’s more compatible with the Pandas python library:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# pull the income statement, balance sheet, and cash flow statement 
# as downloaded from stockrow.com
aapl_inc_df = pd.read_excel(f'./Income/AAPL_inc.xlsx').T
aapl_bal_df = pd.read_excel(f'./BalanceSheet/AAPL_bal.xlsx').T
aapl_cfl_df = pd.read_excel(f'./CashFlow/AAPL_cashflow.xlsx').T

# change the first row to be the column headers
aapl_inc_df.columns = aapl_inc_df.iloc[0]
aapl_bal_df.columns = aapl_bal_df.iloc[0]
aapl_cfl_df.columns = aapl_cfl_df.iloc[0]

# remove the first row
aapl_inc_df = aapl_inc_df.iloc[1:]
aapl_bal_df = aapl_bal_df.iloc[1:]
aapl_cfl_df = aapl_cfl_df.iloc[1:]

# give index a new name
aapl_inc_df=aapl_inc_df.rename_axis("Date",axis=1)
aapl_bal_df=aapl_bal_df.rename_axis("Date",axis=1)
aapl_cfl_df=aapl_cfl_df.rename_axis("Date",axis=1)

# check out the head, just make sure
aapl_inc_df.head(3)

This is *Trailing Twelve Months* or TTM data, which is a way to mimic "annual" numbers on a quarterly basis. We have three unique DataFrames, one corresponding to each of the Income Statement, Balance Sheet, and Cash Flow Statement. 

There are very many fields in the original Excel workbooks, only some of which we’re interested in. So let’s:
1. Identify those columns / variables we’re interested in
2. Extract only those fields
3. Combine the separate DataFrames into 1

I chose an arbitrary set of financial columns, but please explore and see which features you might add or subtract.

In [ ]:
# these are the columns we're interested in
inc_columns = ['Revenue','Gross Profit','Operating Income','Income Tax Provision', 'Net Income Common']
bal_columns = ['Total Assets','Total current assets','Total liabilities','Total current liabilities',
               'Shareholders Equity (Total)','Shares (Common)']
cfl_columns = ['Dividends Paid (Common)','Capital expenditures','Operating Cash Flow','Equity Repurchase (Common, Net)']

# this will apply only the columns above to each of our financial statement DataFrames
aapl_inc_df = aapl_inc_df[inc_columns]
aapl_bal_df = aapl_bal_df[bal_columns]
aapl_cfl_df = aapl_cfl_df[cfl_columns]

# this will combine or concatenate our three DataFrames into one
aapl_df = pd.concat([aapl_inc_df,aapl_bal_df,aapl_cfl_df], axis=1)

aapl_df.head(2)

To recap: we now have clean DataFrames filled with up to 10 years of relevant financial data for AAPL, indexed on a quarterly basis.

## Part II: Calculating growth
Let’s look at Apple’s Revenue over the last 10 years:

In [ ]:
# TTM - Revenue
plt.figure(figsize=(9,6))
sns.scatterplot(x=aapl_df.index,y=aapl_df['Revenue'])
plt.title('Trailing 12 months of Revenue for Apple, Inc')
plt.ylabel('Revenue ($100B)');

Their annual revenues have grown from roughly \\$100B to$300B over the course of a decade. What's the growth rate?

Let's find the most recent one-year change in Revenue with the following code. It's a simple difference formula that indexes 4 quarters apart.

<!--Well, the 1-yr growth rate is different from the 2-yr growth rate and so on.--> 

In [ ]:
# 1-year growth rate, last year of the dataset
(aapl_df['Revenue'][0] - aapl_df['Revenue'][4]) / aapl_df['Revenue'][4]


Extending the formula to 2 years, we follow this algorithm:
- Find the percent difference over the last 8 quarters (t0 minus t8)
- Add 1
- Find tyr1_growth*n*th root (raise power to the (1/**2**) for n = **2** years)
- Subtract 1


In [ ]:
# 2-year growth rate, last year of the dataset
(1 + (aapl_df['Revenue'][0] - aapl_df['Revenue'][8]) / aapl_df['Revenue'][8])**(1/2)-1

There's another method we can use to calculate average growth over the last 10 years. Namely, we iterate through every quarter in the dataset and capture its 1-yr growth rate.

Create a list comprehension to capture each iterated growth rate, then sum all entries and divide by number of entries

In [ ]:
# averaging all 1-yr revenue growths
rev_growth_list = [((aapl_df['Revenue'][i] - aapl_df['Revenue'][i+4]) / aapl_df['Revenue'][i+4]) 
                   for i in range(0,len(aapl_df['Revenue'])-4)]

sum(rev_growth_list)/len(rev_growth_list)

## What about other growth rates?

Let's look at some other metrics:

In [ ]:
#calculating all average growths at once
growth_list = []
for col in aapl_df.columns:
    try:
        col_growth_list = [((aapl_df[col][i] - aapl_df[col][i+4]) / aapl_df[col][i+4]) 
                           for i in range(0,len(aapl_df[col])-4)]
        print(f"Avg growth for {col} is {sum(col_growth_list)/len(col_growth_list)}")
        growth_list.append(sum(col_growth_list)/len(col_growth_list))
    except:
        print(f"{col} throws an error")

print(f"Avg growth: {sum(growth_list)/len(growth_list)}")

##### Sidenote
Based on personal judgment, I’m comfortable removing the lowest values and also the highest values to get a better sense for the ‘typical’ growth rates of AAPL financial metrics:

In [ ]:
# this code sorts the growth rates from low to high, 
# then it removes the top 2 and lowest 2
growth_list.sort()
normalized_aapl_growth = sum(growth_list[2:-2]) / len(growth_list[2:-2])
normalized_aapl_growth

## Part III: Discounted Cash Flow analysis
Now that we have a better sense for what we mean by growth rates, let’s use this to predict a ‘true value’ for Apple, Inc.

We’ll start by creating a new metric, one that doesn’t appear on most balance sheets, called Free Cash Flow (FCF). There are many definitions for FCF, but for expediency we’ll use:

**Operating Cash Flow** minus **Capital Expenditures** equals **FCF**

We create a new DataFrame column for FCF:

In [ ]:
# defining a new column for FCF
aapl_df['FreeCashFlow'] = aapl_df['Operating Cash Flow'] - aapl_df['Capital expenditures']

Using our list comprehension method above, we can determine the average growth rate of FCF over the last decade:

In [ ]:
# averaging all FCF growth rates
fcf_growth_list = [((aapl_df['FreeCashFlow'][i] - aapl_df['FreeCashFlow'][i+4]) / aapl_df['FreeCashFlow'][i+4]) 
                   for i in range(0,len(aapl_df['FreeCashFlow'])-4)]

fcf_growth_rate1 = sum(fcf_growth_list)/len(fcf_growth_list)
print(f"Free Cash Flow average 1-yr growth rate: {round(fcf_growth_rate1*100,2)}%")

The following code projects what next year’s Free Cash Flow might look like, in a series of steps:
- Most recent FCF ($108.7 B)
- Take the most recent FCF and increase by 10.03% (the avg. FCF growth rate)
- Take the above and *decrease* it by the discount rate 1.63%

I chose a discount rate of 1.63\% because it’s equal to the current interest rates on 10-year Treasury Bills. This is purely for expediency; you might use weighted-average cost of capital (WACC) or some other discount rate methodology. Here’s the code:

In [ ]:
# Calculate most recent FCF, next year FCF, and discount next year's FCF
bill = 1_000_000_000
discount_rate = 0.0163

print(f"Most recent FCF: ${round(aapl_df['FreeCashFlow'][0]/bill,2)} billion")
print(f"Next year FCF: ${round(aapl_df['FreeCashFlow'][0]*(1+fcf_growth_rate1)/bill,2)} billion")
print(f"Next year FCF, discounted: ${round(aapl_df['FreeCashFlow'][0]*(1+fcf_growth_rate1)**(1)/(1+discount_rate)/bill,2)} billion")

Now that we know how to project next year’s discounted FCF, we can naturally extend the analysis to year 2, year 3, and so on. Let’s assume the growth rates for AAPL’s next 10 years will be similar to the last 10. We’ll calculate FCF for each year, then add them all together:

In [ ]:
# DCF using the treasury rate as the discount rate and avg. FCF growth as growth rate
treasury_rate = 0.0163
# we start at zero, then incrementally add each subsequent year's FCF
fcf_over_time1 = 0
# our base will be the most recent year's FCF
fcf_start = aapl_df['FreeCashFlow'][0]

# covering a range of 10 years
for i in range(1,10+1):
    fcf_over_time1 += fcf_start * (1+fcf_growth_rate1)**i / (1+treasury_rate)**i
    print(f"Year {i}: ${round(fcf_start * (1+fcf_growth_rate1)**i / (1+treasury_rate)**i / 1_000_000_000,1)} billion")

print(f"Total FCF: ${round(fcf_over_time1/1_000_000_000,1)} billion")

$1.7 trillion, wow! What about 15 years? Let’s try it! 

Let’s also increase our discount rate to 5%, which will reduce the final value — and that makes sense because it’s so very difficult to project 15 years ahead of time:

In [ ]:
# DCF using 5% as the discount rate and avg. FCF growth as growth rate
discount_rate = 0.05
# we start at zero, then incrementally add each subsequent year's FCF
fcf_over_time = 0
# our base will be the most recent year's FCF
fcf_start = aapl_df['FreeCashFlow'][0]

# covering a range of 15 years
for i in range(1,15+1):
    fcf_over_time += fcf_start * (1+fcf_growth_rate1)**i / (1+discount_rate)**i
    print(f"Year {i}: ${round(fcf_start * (1+fcf_growth_rate1)**i / (1+discount_rate)**i / 1_000_000_000,1)} billion")

print(f"Total FCF: ${round(fcf_over_time/1_000_000_000,1)} billion")

##### $2.4 trillion\!
How much FCF per share of AAPL common stock?

In [ ]:
# turning FCF into per-share stock price predictions
aapl_fcf1 = round(fcf_over_time1/aapl_df['Shares (Common)'][0],2)
aapl_fcf2 = round(fcf_over_time/aapl_df['Shares (Common)'][0],2)

avg_fcf = round((aapl_fcf1 + aapl_fcf2)/2,2)

print(f"Total FCF per share, v1: ${aapl_fcf1}")
print(f"Total FCF per share, v2: ${aapl_fcf2}")
print(f"Average FCF per share: ${avg_fcf}")